### Setup

In [1]:
import os
from typing import Dict, List
from groq import Groq

In [ ]:
LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
LLAMA3_8B_INSTRUCT = "llama3.1-8b-instant"

In [ ]:
DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

In [ ]:
TEMPERATURE = 0.0
TOP_P = 1.0

In [ ]:
client = Groq()

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = TEMPERATURE,
    top_p: float = TOP_P,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content


def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = TEMPERATURE,
    top_p: float = TOP_P,
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    response = completion(prompt, model)
    print(response, end='\n\n')

### Working Prompt

In [ ]:
TEST_PROMPT="""
False Sharing Bug Detection and Resolution

Role: C Code Analyzer and Optimizer

Task Description:
Detect and repair false sharing bugs in C/C++ code.

Definition:
False sharing: Occurs when multiple threads access distinct variables or disjoint bytes in the same cache line. They do not access the same
memory location or overlapping bytes.

Guidelines for Detecting False Sharing:
1. Memory Proximity: Look for variables or fields that are close together in memory (e.g., in a contiguous array or in the same struct).
2. Multiple Threads: Detect variables accessed by multiple threads where at least one thread writes to the variable.
3. Cache Line Size: Analyze memory layout with respect to the cache line size.

Guidelines for Correction (False Sharing Only):
1. Apply alignment and padding to separate variables and avoid placing them in the same cache line.
2. Use scratch variables or intermediate results to reduce writes to the cache line.

Chain of Thought:
1. Analyze the code structure based on the Guidelines for Detecting False Sharing.
2. If a false sharing bug is detected, then proceed to step 3. Otherwise, proceed to step 4.
3. Generate a corrected code snippet using the Guidelines for Correction and clearly indicate which lines were modified. Then, proceed to step 4.
4. If you have not reached the end of the code, continue analyzing the rest of the code for any additional false sharing bugs. Otherwise, conclude the analysis.


Example 1 (False Sharing):

#include <pthread.h>
#include <stdio.h>

#define THREADS 4

typedef struct {
    int a;
    int b;
} data_t;

data_t data[THREADS];

void* worker(void* arg) {
    int index = *(int*)arg;
    for (int i = 0; i < 1000000; i++) {
        data[index].a += 1;
        data[index].b += 2;
    }
    return NULL;
}

int main() {
    pthread_t threads[THREADS];
    int indices[THREADS];

    for (int i = 0; i < THREADS; i++) {
        indices[i] = i;
        pthread_create(&threads[i], NULL, worker, &indices[i]);
    }

    for (int i = 0; i < THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    return 0;
}

Chain of Thought:
1.Memory Proximity: The variables a and b are close in memory, as they are part of the same data_t structure and stored in a contiguous array.
2.Multiple Threads: Each thread operates on a distinct index of the data array. The threads do not access the same memory, but they access adjacent fields within the same structure.
3. Cache Line Size: If a and b fall within the same cache line (typically 64 bytes), this could lead to false sharing as multiple threads are writing to different parts of the same cache line.

Bug Classification:
False Sharing Detected: Multiple threads are accessing disjoint bytes of the same struct in the same cache line, so this false sharing.

Corrected Code:

#include <pthread.h>
#include <stdio.h>

#define THREADS 4

typedef struct {
    int a;
    int b;
    char padding[64]; // Add padding to avoid false sharing
} data_t;

data_t data[THREADS];

void* worker(void* arg) {
    int index = *(int*)arg;
    for (int i = 0; i < 1000000; i++) {
        data[index].a += 1;
        data[index].b += 2;
    }
    return NULL;
}

int main() {
    pthread_t threads[THREADS];
    int indices[THREADS];

    for (int i = 0; i < THREADS; i++) {
        indices[i] = i;
        pthread_create(&threads[i], NULL, worker, &indices[i]);
    }

    for (int i = 0; i < THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    return 0;
}

Explanation: Padding is added to the struct to separate a and b into distinct cache lines, reducing the chance of false sharing.


Example 2 (Not False Sharing):

#include <pthread.h>
#include <stdio.h>

#define THREADS 4

int shared_sum = 0;

void* worker(void* arg) {
    for (int i = 0; i < 1000000; i++) {
        shared_sum++; // Multiple threads write to the same memory location
    }
    return NULL;
}

int main() {
    pthread_t threads[THREADS];

    for (int i = 0; i < THREADS; i++) {
        pthread_create(&threads[i], NULL, worker, NULL);
    }

    for (int i = 0; i < THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    printf("Final sum: %d\n", shared_sum);

    return 0;
}

Chain of Thought:
1.Memory Proximity: Only one shared variable, shared_sum, is being accessed by multiple threads.
2.Multiple Threads: All threads increment the same shared_sum variable.
3.Cache Line Size: Not relevant, as threads access the exact same memory location.

Bug Classification:
No False Sharing Detected: All threads are writing to the same memory location in the cache line. Since the threads are not accessing disjoint bytes, this is not a false sharing bug.

Corrected Code:
No correction needed, since this code does not have false sharing. The use of proper synchronization mechanisms would resolve any concurrency issues.


Your Task:
Evaluate the following code example for false sharing bugs and provide corrected code if necessary.

Code Example:
{}

Task Requirements:
1. Determine whether the provided code contains a false sharing bug or not.
2. Only if a false sharing is present then provide code corrections using alignment and padding techniques.
3. Rewrite only the modified code sections and provide line numbers that indicate which lines of the original code were modified.

Evaluation Criteria:
- Accuracy in detecting false sharing bugs
- Efficiency of generated code snippets

Additional Instructions for Performing Task:
- Focus on identifying and correcting false sharing bugs.
- Limit output to rewritten code sections.
- Provide clear and concise explanations.
- Import <stdalign.h> library when using alignas in corrected code solutions.
- Double check your generated code to ensure you have not unnecessarily changed the original code.
- When given an example, number each line in the code. When you provide a corrected code solution, indicate which lines were modified.
- if contiguous memory is accessing memory across multiple cache lines, that is not false sharing.
"""

In [ ]:
histogram_case = """
/* Copyright (c) 2007, Stanford University
* All rights reserved.
*
* Redistribution and use in source and binary forms, with or without
* modification, are permitted provided that the following conditions are met:
*     * Redistributions of source code must retain the above copyright
*       notice, this list of conditions and the following disclaimer.
*     * Redistributions in binary form must reproduce the above copyright
*       notice, this list of conditions and the following disclaimer in the
*       documentation and/or other materials provided with the distribution.
*     * Neither the name of Stanford University nor the
*       names of its contributors may be used to endorse or promote products
*       derived from this software without specific prior written permission.
*
* THIS SOFTWARE IS PROVIDED BY STANFORD UNIVERSITY ``AS IS'' AND ANY
* EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
* WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
* DISCLAIMED. IN NO EVENT SHALL STANFORD UNIVERSITY BE LIABLE FOR ANY
* DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
* (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
* LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
* ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
* (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
* SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
*/

#include <stdio.h>
#include <strings.h>
#include <string.h>
#include <stddef.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <ctype.h>
#include <pthread.h>

#include "stddefines.h"

#define IMG_DATA_OFFSET_POS 10
#define BITS_PER_PIXEL_POS 28

int swap;      // to indicate if we need to swap byte order of header information

typedef struct {
   unsigned char *data;
   long data_pos;
   long data_len;
   int red[256];
   int green[256];
   int blue[256];
   //char padding[40];
} thread_arg_t;

/* test_endianess
 *
 */
void test_endianess() {
   unsigned int num = 0x12345678;
   char *low = (char *)(&(num));
   if (*low ==  0x78) {
      dprintf("No need to swap\n");
      swap = 0;
   }
   else if (*low == 0x12) {
      dprintf("Need to swap\n");
      swap = 1;
   }
   else {
      printf("Error: Invalid value found in memory\n");
      exit(1);
   }
}

/* swap_bytes
 *
 */
void swap_bytes(char *bytes, int num_bytes) {
   int i;
   char tmp;

   for (i = 0; i < num_bytes/2; i++) {
      dprintf("Swapping %d and %d\n", bytes[i], bytes[num_bytes - i - 1]);
      tmp = bytes[i];
      bytes[i] = bytes[num_bytes - i - 1];
      bytes[num_bytes - i - 1] = tmp;
   }
}

/* calc_hist
 * Function that computes the histogram for the region
 * assigned to each thread
 */
void *calc_hist(void *arg) {

   int *red;
   int *green;
   int *blue;
   int i,j;
   thread_arg_t *thread_arg = (thread_arg_t *)arg;
   unsigned char *val;
   /*
   red = (int *)calloc(256, sizeof(int));
   green = (int *)calloc(256, sizeof(int));
   blue = (int *)calloc(256, sizeof(int));
   */
   red = thread_arg->red;
   green = thread_arg->green;
   blue = thread_arg->blue;


   //printf("Starting at %ld, doing %ld bytes\n", thread_arg->data_pos, thread_arg->data_len);
   for(j=0; j<60; j++){
   for (i= thread_arg->data_pos;
        i < thread_arg->data_pos + thread_arg->data_len;
        i+=3) {

      val = &(thread_arg->data[i]);
      blue[*val]++;

      val = &(thread_arg->data[i+1]);
      green[*val]++;

      val = &(thread_arg->data[i+2]);
      red[*val]++;
   }
   }
   /*
   thread_arg->red = red;
   thread_arg->green = green;
   thread_arg->blue = blue;
   */
   return (void *)0;
}


int main(int argc, char *argv[]) {

   int i, j;
   int fd;
   char *fdata;
   struct stat finfo;
   char * fname;
   pthread_t *pid;
   pthread_attr_t attr;
   thread_arg_t *arg;
   int red[256];
   int green[256];
   int blue[256];
   int num_procs = 4;
   int num_per_thread;
   int excess;


   // Make sure a filename is specified
   if (argv[1] == NULL) {
      printf("USAGE: %s <bitmap filename>\n", argv[0]);
      exit(1);
   }

   fname = argv[1];

   // Read in the file
   CHECK_ERROR((fd = open(fname, O_RDONLY)) < 0);
   // Get the file info (for file length)
   CHECK_ERROR(fstat(fd, &finfo) < 0);
   // Memory map the file
   CHECK_ERROR((fdata = mmap(0, finfo.st_size + 1,
      PROT_READ | PROT_WRITE, MAP_PRIVATE, fd, 0)) == NULL);

   if ((fdata[0] != 'B') || (fdata[1] != 'M')) {
      printf("File is not a valid bitmap file. Exiting\n");
      exit(1);
   }

   test_endianess();    // will set the variable "swap"

   unsigned short *bitsperpixel = (unsigned short *)(&(fdata[BITS_PER_PIXEL_POS]));
   if (swap) {
      swap_bytes((char *)(bitsperpixel), sizeof(*bitsperpixel));
   }
   if (*bitsperpixel != 24) {    // ensure its 3 bytes per pixel
      printf("Error: Invalid bitmap format - ");
      printf("This application only accepts 24-bit pictures. Exiting\n");
      exit(1);
   }

   unsigned short *data_pos = (unsigned short *)(&(fdata[IMG_DATA_OFFSET_POS]));
   if (swap) {
      swap_bytes((char *)(data_pos), sizeof(*data_pos));
   }

   int imgdata_bytes = (int)finfo.st_size - (int)(*(data_pos));
   int num_pixels = ((int)finfo.st_size - (int)(*(data_pos))) / 3;
   printf("This file has %d bytes of image data, %d pixels\n", imgdata_bytes,
                                                            num_pixels);

   printf("Starting pthreads histogram\n");


   memset(&(red[0]), 0, sizeof(int) * 256);
   memset(&(green[0]), 0, sizeof(int) * 256);
   memset(&(blue[0]), 0, sizeof(int) * 256);

   /* Set a global scope */
   pthread_attr_init(&attr);
   pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);

   //CHECK_ERROR((num_procs = sysconf(_SC_NPROCESSORS_ONLN)) <= 0);
   num_per_thread = num_pixels / num_procs;
   excess = num_pixels % num_procs;

   CHECK_ERROR( (pid = (pthread_t *)malloc(sizeof(pthread_t) * num_procs)) == NULL);
   CHECK_ERROR( (arg = (thread_arg_t *)malloc(sizeof(thread_arg_t)* num_procs)) == NULL);
   memset(arg, 0, sizeof(thread_arg_t)*num_procs);
   //printf("%p\n",arg);

   /* Assign portions of the image to each thread */
   long curr_pos = (long)(*data_pos);
   for (i = 0; i < num_procs; i++) {
      arg[i].data = (unsigned char *)fdata;
      arg[i].data_pos = curr_pos;
      long tmp_data_len = num_per_thread;
      if (excess > 0) {
         tmp_data_len++;
         excess--;
      }
      arg[i].data_len = tmp_data_len;

      arg[i].data_len *= 3;   // 3 bytes per pixel
      curr_pos += arg[i].data_len;

      pthread_create(&(pid[i]), &attr, calc_hist, (void *)(&(arg[i])));
   }

   for (i = 0; i < num_procs; i++) {
      pthread_join(pid[i] , NULL);
   }

   for (i = 0; i < num_procs; i++) {
      for (j = 0; j < 256; j++) {
         red[j] += arg[i].red[j];
         green[j] += arg[i].green[j];
         blue[j] += arg[i].blue[j];
      }
   }

   dprintf("\n\nBlue\n");
   dprintf("----------\n\n");
   for (i = 0; i < 256; i++) {
      dprintf("%d - %d\n", i, blue[i]);
   }

   dprintf("\n\nGreen\n");
   dprintf("----------\n\n");
   for (i = 0; i < 256; i++) {
      dprintf("%d - %d\n", i, green[i]);
   }

   dprintf("\n\nRed\n");
   dprintf("----------\n\n");
   for (i = 0; i < 256; i++) {
      dprintf("%d - %d\n", i, red[i]);
   }

   CHECK_ERROR(munmap(fdata, finfo.st_size + 1) < 0);
   CHECK_ERROR(close(fd) < 0);

   free(pid);
   /*for(i = 0; i < num_procs; i++) {
      free(arg[i].red);
      free(arg[i].green);
      free(arg[i].blue);
   }*/
   //free(arg);
   pthread_attr_destroy(&attr);

   return 0;
}

"""

In [ ]:
complete_and_print(TEST_PROMPT.replace("{}", histogram_case))